This content was scrapped on **"Pakistan"** origing based news websites including 

**"DAWN NEWS"**, **"THE NATION NEWS"**, & **"THE NEWS"** in which i prefer to target their

**"Opinions"** section in which i scrapped **"Editorials"**, **"Columns"**, & **"Letters to the editors"** of these news paper from

**"JANUARY 1st, 2008"** to **"MARCH 10th, 2024"** which consists a huge number of content from these articles.

Here is a short description of the content:

**COLUMNS:** Dawn News (**29531**), The Nation News (**37937**), THE NEWS (**32562**)

**EDITORIALS:** Dawn News (**24161**), The Nation News (**35928**), THE NEWS (**21598**)

**LETTERS TO THE EDITORS:** Dawn News (**23895**), The Nation News (**12428**), THE NEWS (**34298**)

*******************************************************************************************************************************************************

**Important Libraries**

In [1]:
import requests  # Importing the requests library to make HTTP requests
from bs4 import BeautifulSoup  # Importing BeautifulSoup for HTML parsing
import csv  # Importing csv module for handling CSV files
from selenium import webdriver  # Importing the Selenium webdriver for browser automation
from datetime import datetime, timedelta  # Importing datetime and timedelta for working with dates
import pandas as pd  # Importing pandas for data manipulation and analysis


**Dawn News Scraper**

In [ ]:
class DawnNews:
    def __init__(self, url, headers):
        self.url = url  # Assigning the URL of the DawnNews website to the object's url attribute
        self.headers = headers  # Assigning the headers to be used for making HTTP requests to the object's headers attribute
        self.driver = webdriver.Chrome()  # Initializing a Chrome webdriver instance
        
    def parse_articles(self):
        try:
            response = requests.get(self.url, headers=self.headers)  # Sending an HTTP GET request to the provided URL with the specified headers
            # print(response.content)
            response.raise_for_status()  # Checking for any HTTP request errors
            soup = BeautifulSoup(response.text, 'lxml')  # Creating a BeautifulSoup object to parse the HTML content of the response
            articles = (soup.find_all('article', {'class': 'jeg_post jeg_pl_sm format-standard column-category-border-wrapper'}))  # Finding all articles on the page with the specified class
            
            for article in articles:  # Iterating over each article found
                columnistLink = article.find('div', {'class': 'jeg_thumb'}).find('a')['href']  # Extracting the link of the columnist
                columnTitle = article.find('h3', {'class': 'jeg_post_title'}).find('a').text  # Extracting the title of the article
                columnLink = article.find('h3', {'class': 'jeg_post_title'}).find('a')['href']  # Extracting the link of the article
                columnistName = article.find('div', {'class': 'jeg_meta_author'}).find('a').text  # Extracting the name of the columnist
                columPublishDate = article.find('div', {'class': 'jeg_meta_date'}).text  # Extracting the publication date of the article
                
                # Printing out the extracted information for each article
                print(columnistName, columnTitle, columPublishDate, columnLink, columnistLink)
        
        except requests.exceptions.RequestException as e:  # Catching any HTTP request exceptions
            print(f'Error: {e}')  # Printing out the error message if an exception occurs

            
    # def scrap_columns_section_list(self):
    #     try:
    #         self.driver.get(self.url)
    #         wait = WebDriverWait(self.driver, 3)
    #         load_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Load More")]')))

    #         while load_more_button:
    #             load_more_button.click()
    #             # Wait for new articles to load
    #             wait.until(EC.presence_of_all_elements_located((By.XPATH, '//article[@class="jeg_post jeg_pl_sm format-standard column-category-border-wrapper"]')))
    #             soup = BeautifulSoup(self.driver.page_source, 'lxml')
    #             self.parse_articles(soup)
                
    #             # Check if there are more pages to load
    #             load_more_button = self.driver.find_element_by_xpath('//button[contains(text(), "Load More")]')

    #     finally:
    #         self.driver.quit()
        

In [ ]:
DawnObject = DawnNews(  # Creating an instance of the DawnNews class with the specified parameters
                        'https://www.nation.com.pk/columns',  # Providing the URL of the DawnNews website to scrape
                        headers={  # Providing the headers to be used for making HTTP requests
                              'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',  # User-Agent header to simulate a web browser
                              'Accept-Language': 'en-US,en;q=0.9',  # Accept-Language header to specify preferred languages for content
                              'Accept-Encoding': 'gzip, deflate, br',  # Accept-Encoding header for specifying acceptable content encoding
                              'Connection': 'keep-alive',  # Connection header to keep the connection open
                        }
                  )


In [ ]:
DawnObject.parse_articles()  # Calling the parse_articles method of the DawnObject instance
# DawnObject.scrap_columns_section_list()  # This line is commented out and not executed because the method is not implemented yet

**The Nation News Column Scrapper**

In [ ]:
class NationScraper:
    def __init__(self, url, headers):
        self.base_url = url  # Assigning the base URL of the Nation website to the object's base_url attribute
        self.headers = headers  # Assigning the headers to be used for making HTTP requests to the object's headers attribute

    def generate_date_range(self, start_date, end_date):
        # Generating a range of dates between start_date and end_date
        start = datetime.strptime(start_date, '%d-%b-%Y')  # Converting the start_date string to a datetime object
        end = datetime.strptime(end_date, '%d-%b-%Y')  # Converting the end_date string to a datetime object
        delta = timedelta(days=1)  # Defining a timedelta object representing one day
        while start <= end:  # Looping through each date in the range
            yield start.strftime('%d-%b-%Y')  # Yielding the current date formatted as 'dd-MMM-YYYY'
            start += delta  # Moving to the next date in the range

    def scrap_columns_by_date(self, date):
        # Scraping columns for a specific date
        url = f"{self.base_url}{date}/columns"  # Constructing the URL for the specific date
        try:
            response = requests.get(url, headers=self.headers)  # Sending an HTTP GET request to the constructed URL with the specified headers
            response.raise_for_status()  # Checking for any HTTP request errors
            soup = BeautifulSoup(response.text, 'html.parser')  # Creating a BeautifulSoup object to parse the HTML content of the response
            self.parse_articles(soup, date)  # Parsing the articles from the response
        except requests.exceptions.RequestException as e:  # Handling any HTTP request exceptions
            print(f"Error scraping data for {date}: {e}")  # Printing out an error message if an exception occurs

    def parse_articles(self, soup, date):
        # Parsing articles from the HTML content
        articles_list = []  # Creating an empty list to store information about articles
        articles = (soup.find_all('article', {'class': 'jeg_post jeg_pl_sm format-standard column-category-border-wrapper'}))  # Finding all articles on the page with the specified class
        if articles:  # Checking if articles are found for the given date
            print(f"Articles for {date}:")  # Printing the date for which articles are being parsed
            for article in articles:  # Iterating over each article found
                # Initializing variables with default values
                columnistLink = 'NF'
                columnTitle = 'NF'
                columnLink = 'NF'
                columnistName = 'NF'
                columPublishDate = 'NF'

                # Extracting information from various HTML elements within the article
                thumb_div = article.find('div', {'class': 'jeg_thumb'})
                if thumb_div:
                    link_a = thumb_div.find('a')
                    if link_a and 'href' in link_a.attrs:
                        columnistLink = link_a['href']

                title_h3 = article.find('h3', {'class': 'jeg_post_title'})
                if title_h3:
                    link_a = title_h3.find('a')
                    if link_a and 'href' in link_a.attrs:
                        columnLink = link_a['href']
                    columnTitle = link_a.text

                author_div = article.find('div', {'class': 'jeg_meta_author'})
                if author_div:
                    author_a = author_div.find('a')
                    if author_a:
                        columnistName = author_a.text

                date_div = article.find('div', {'class': 'jeg_meta_date'})
                if date_div:
                    columPublishDate = date_div.text
                
                # Adding extracted information to the articles_list
                articles_list.append({
                    'columnistName': columnistName,
                    'columnTitle': columnTitle,
                    'columPublishDate': columPublishDate,
                    'columnLink': columnLink,
                    'columnistLink': columnistLink
                })
                
                # Printing out information about each article
                print(columnistName, columnTitle, columPublishDate, columnLink, columnistLink)
                
                # Saving articles to a CSV file
                self.save_to_csv(articles_list)
        else:
            print(f"No column found for {date}")  # Printing a message if no articles are found for the given date
    
    def save_to_csv(self, articles_list):
        # Saving articles to a CSV file
        with open('nation_columns.csv', mode='a', newline='', encoding='utf-8') as file:  # Opening the CSV file in append mode
            fieldnames = ['columnistName', 'columnTitle', 'columPublishDate', 'columnLink', 'columnistLink']  # Defining field names for CSV columns
            writer = csv.DictWriter(file, fieldnames=fieldnames)  # Creating a CSV DictWriter object
            if file.tell() == 0:  # Checking if the file pointer is at the beginning of the file (empty file)
                writer.writeheader()  # Writing header row if the file is empty
            for article in articles_list:  # Iterating over each article in the articles_list
                writer.writerow(article)  # Writing article information to the CSV file row by row

    def scrape_date_range(self, start_date, end_date):
        # Scraping columns for a range of dates
        for date in self.generate_date_range(start_date, end_date):  # Iterating over each date in the generated date range
            self.scrap_columns_by_date(date)  # Scraping columns for the current date


In [ ]:
NationObject = NationScraper(  # Creating an instance of the NationScraper class with the specified parameters
    'https://www.nation.com.pk/archives/',  # Providing the base URL of the Nation website to scrape
    headers={  # Providing the headers to be used for making HTTP requests
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',  # User-Agent header to simulate a web browser
        'Accept-Language': 'en-US,en;q=0.9',  # Accept-Language header to specify preferred languages for content
        'Accept-Encoding': 'gzip, deflate, br',  # Accept-Encoding header for specifying acceptable content encoding
        'Connection': 'keep-alive',  # Connection header to keep the connection open
    }
)

# start_date = '01-Jan-2008'
# end_date = '10-Mar-2024'
start_date = '08-Mar-2024'  # Setting the start date for the scraping range
end_date = '10-Mar-2024'  # Setting the end date for the scraping range

NationObject.scrape_date_range(start_date, end_date)  # Calling the scrape_date_range method of the NationObject instance

In [ ]:
# Creating an instance of the NationScraper class with the specified parameters
NationObject = NationScraper(
    'https://www.nation.com.pk/archives/',  # Providing the base URL of the Nation website to scrape
    headers={  # Providing the headers to be used for making HTTP requests
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',  # User-Agent header to simulate a web browser
        'Accept-Language': 'en-US,en;q=0.9',  # Accept-Language header to specify preferred languages for content
        'Accept-Encoding': 'gzip, deflate, br',  # Accept-Encoding header for specifying acceptable content encoding
        'Connection': 'keep-alive',  # Connection header to keep the connection open
    }
)

# Defining the start and end dates for the scraping range
start_date = '10-Jan-2012'  # Setting the start date for the scraping range
end_date = '10-Mar-2024'  # Setting the end date for the scraping range

# Initiating the scraping process for articles published within the specified date range on the Nation website
NationObject.scrape_date_range(start_date, end_date)


**Preprocessing for articles**

In [ ]:
# Reading data from a CSV file named 'articles.csv' into a pandas DataFrame
df = pd.read_csv('articles.csv')

# Displaying the first few rows of the DataFrame
df.head()


In [ ]:
# Checking for duplicate rows in the DataFrame and summing up the count of duplicates
df.duplicated().sum()

In [ ]:
# Removing duplicate rows from the DataFrame and assigning the result back to df
df = df.drop_duplicates()

# Checking for duplicate rows in the DataFrame after removing duplicates and summing up the count of duplicates
df.duplicated().sum()

In [ ]:
# Displaying the first few rows of the DataFrame to inspect the changes after removing duplicates
df.head()

**The Nation News Column Content Scrapper**

In [ ]:
class ArticleScraper:
    def __init__(self, csv_file):
        # Initializing the ArticleScraper object with the CSV file to read and write data
        self.csv_file = csv_file
        # Setting headers to be used for making HTTP requests
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
        }

    def scrape_articles(self):
        # Opening the CSV file for reading data
        with open(self.csv_file, mode='r', newline='', encoding='utf-8') as file:
            # Creating a DictReader object to read rows from the CSV file as dictionaries
            reader = csv.DictReader(file)
            # Getting fieldnames from the reader and adding 'articleContent' as a new field
            fieldnames = reader.fieldnames + ['articleContent']
            rows = []
            # Iterating over each row in the CSV file
            for row in reader:
                # Extracting the article URL from the row
                article_url = row['columnLink']
                # Scraping article content for the article URL
                article_content = self.scrape_article_content(article_url)
                # Adding the scraped article content to the row
                row['articleContent'] = article_content
                # Adding the modified row to the list of rows
                rows.append(row)
                # Printing the modified row
                print(row)

        # Opening the CSV file for writing data
        with open(self.csv_file, mode='w', newline='', encoding='utf-8') as file:
            # Creating a DictWriter object to write rows to the CSV file
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            # Writing the header row to the CSV file
            writer.writeheader()
            # Writing the modified rows to the CSV file
            writer.writerows(rows)

    def scrape_article_content(self, article_url):
        try:
            # Sending an HTTP GET request to the article URL with specified headers
            response = requests.get(article_url, headers=self.headers)
            # Checking for any HTTP request errors
            response.raise_for_status()
            # Creating a BeautifulSoup object to parse the HTML content of the response
            soup = BeautifulSoup(response.text, 'html.parser')

            # Scraping logic for article content, modify as needed
            article_content = 'NP'  # Default value for article content
            # Finding the HTML element containing the article content
            content = soup.find('div', {'class': 'content-inner news-detail-content-class'})
            # Extracting the text content if found
            if content:
                article_content = content.text

            return article_content  # Returning the scraped article content

        except requests.exceptions.RequestException as e:
            # Handling any HTTP request exceptions
            print(f"Error scraping content from {article_url}: {e}")
            return ''  # Returning an empty string if scraping fails

In [ ]:
# Example usage

# Define the name of the CSV file containing article information
csv_file = 'nation_columns.csv'

# Create an instance of the ArticleScraper class, providing the CSV file name as an argument
scraper = ArticleScraper(csv_file)

# Call the scrape_articles() method of the ArticleScraper instance to initiate the scraping process
scraper.scrape_articles()


**The Nation News Editorials Scrapper**

In [ ]:
class NationScraper:
    def __init__(self, url, headers):
        # Initialize NationScraper object with base URL and headers
        self.base_url = url
        self.headers = headers

    def generate_date_range(self, start_date, end_date):
        # Generate a range of dates between start_date and end_date
        start = datetime.strptime(start_date, '%d-%b-%Y')
        end = datetime.strptime(end_date, '%d-%b-%Y')
        delta = timedelta(days=1)
        while start <= end:
            yield start.strftime('%d-%b-%Y')
            start += delta

    def scrap_editorials_by_date(self, date):
        # Scrape editorials for a specific date
        url = f"{self.base_url}{date}/editorials"
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            self.parse_editorials(soup, date)
        except requests.exceptions.RequestException as e:
            print(f"Error scraping data for {date}: {e}")

    def parse_editorials(self, soup, date):
        # Parse editorials from the HTML content
        editorials_list = []
        articles = (soup.find_all('article', {'class': 'jeg_post jeg_pl_md_1 format-standard'}))
        if articles:
            print(f"Editorials for {date}:")
            for article in articles:
                editorialLink = 'NF'  # Default value for editorial link
                editorialName = 'NF'  # Default value for editorial name
                editorialPublishDate = 'NF'  # Default value for editorial publish date

                # Extracting editorial link
                thumb_div = article.find('div', {'class': 'jeg_thumb'})
                if thumb_div:
                    link_a = thumb_div.find('a')
                    if link_a and 'href' in link_a.attrs:
                        editorialLink = link_a['href']

                # Extracting editorial name
                title_h3 = article.find('div', {'class': 'jeg_postblock_content'})
                if title_h3:
                    link_a = title_h3.find('a')
                    if link_a and 'href' in link_a.attrs:
                        editorialName = link_a.text

                # Extracting editorial publish date
                date_div = article.find('div', {'class': 'jeg_meta_date'})
                if date_div:
                    editorialPublishDate = date_div.text
                
                # Adding extracted information to the editorials_list
                editorials_list.append({
                    'editorialName': editorialName,
                    'editorialPublishDate': editorialPublishDate,
                    'editorialLink': editorialLink,
                })
                
                # Printing out information about each editorial
                print(editorialName, editorialPublishDate, editorialLink)
                
                # Saving editorials to a CSV file
                self.save_to_csv(editorials_list)
        else:
            print(f"No editorial found for {date}")
    
    def save_to_csv(self, articles_list):
        # Saving editorials to a CSV file
        with open('nation_editorials.csv', mode='a', newline='', encoding='utf-8') as file:
            fieldnames = ['editorialName', 'editorialPublishDate', 'editorialLink']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            if file.tell() == 0:
                writer.writeheader()
            for article in articles_list:
                writer.writerow(article)

    def scrape_date_range(self, start_date, end_date):
        # Scrape editorials for a range of dates
        for date in self.generate_date_range(start_date, end_date):
            self.scrap_editorials_by_date(date)


In [ ]:
# Creating an instance of the NationScraper class with the specified parameters
NationObject = NationScraper(
    'https://www.nation.com.pk/archives/',  # Providing the base URL of the Nation website to scrape
    headers={  # Providing the headers to be used for making HTTP requests
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',  # User-Agent header to simulate a web browser
        'Accept-Language': 'en-US,en;q=0.9',  # Accept-Language header to specify preferred languages for content
        'Accept-Encoding': 'gzip, deflate, br',  # Accept-Encoding header for specifying acceptable content encoding
        'Connection': 'keep-alive',  # Connection header to keep the connection open
    }
)

start_date = '01-Jan-2008'  # Setting the start date for the scraping range
end_date = '10-Mar-2024'  # Setting the end date for the scraping range

# Initiating the scraping process for editorials published within the specified date range on the Nation website
NationObject.scrape_date_range(start_date, end_date)

**Preprocessing for the Nation News article**

In [ ]:
# Reading data from a CSV file named 'nation_editorials.csv' into a pandas DataFrame
df = pd.read_csv('nation_editorials.csv')

# Displaying the first few rows of the DataFrame
df.head()


In [ ]:
# Checking for duplicate rows in the DataFrame and summing up the count of duplicates
df.duplicated().sum()

In [ ]:
# Removing duplicate rows from the DataFrame and assigning the result back to df
df = df.drop_duplicates()

# Checking for duplicate rows in the DataFrame after removing duplicates and summing up the count of duplicates
df.duplicated().sum()

In [ ]:
# Displaying the first few rows of the DataFrame to inspect the changes after removing duplicates
df.head()

In [ ]:
# Filtering out rows where the 'editorialName' column contains the word 'Cartoon'
df = df[~df['editorialName'].str.contains('Cartoon')]

# Displaying the first few rows of the filtered DataFrame
df.head()

**The Nation News Editorials Content Scrapper**

In [ ]:
class ArticleScraper:
    def __init__(self, csv_file):
        # Initialize the ArticleScraper object with the CSV file name and headers for HTTP requests
        self.csv_file = csv_file
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
        }

    def scrape_articles(self):
        # Open the CSV file for reading
        with open(self.csv_file, mode='r', newline='', encoding='utf-8') as file:
            # Create a DictReader object to read rows from the CSV file as dictionaries
            reader = csv.DictReader(file)
            # Define fieldnames for the output CSV file including a new field 'editorialContent'
            fieldnames = reader.fieldnames + ['editorialContent']
            rows = []
            # Iterate over each row in the CSV file
            for row in reader:
                # Extract the article URL from the row
                article_url = row['editorialLink']
                # Scrape the article content for the article URL
                article_content = self.scrape_article_content(article_url)
                # Add the scraped article content to the row
                row['editorialContent'] = article_content
                # Append the modified row to the list of rows
                rows.append(row)
                # Print the modified row
                print(row)

        # Open the CSV file for writing
        with open(self.csv_file, mode='w', newline='', encoding='utf-8') as file:
            # Create a DictWriter object to write rows to the CSV file
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            # Write the header row to the CSV file
            writer.writeheader()
            # Write the modified rows to the CSV file
            writer.writerows(rows)

    def scrape_article_content(self, article_url):
        try:
            # Send an HTTP GET request to the article URL with specified headers
            response = requests.get(article_url, headers=self.headers)
            # Check for any HTTP request errors
            response.raise_for_status()
            # Create a BeautifulSoup object to parse the HTML content of the response
            soup = BeautifulSoup(response.text, 'html.parser')

            # Scraping logic for article content, modify as needed
            article_content = 'NP'  # Default value for article content
            # Find the HTML element containing the article content
            content = soup.find('div', {'class': 'content-inner news-detail-content-class'})
            # Extract the text content if found
            if content:
                article_content = content.text

            return article_content  # Return the scraped article content

        except requests.exceptions.RequestException as e:
            # Handle any HTTP request exceptions
            print(f"Error scraping content from {article_url}: {e}")
            return ''  # Return an empty string if scraping fails


In [ ]:
# Example usage

# Define the name of the CSV file containing editorial information
csv_file = 'nation_editorials.csv'

# Create an instance of the ArticleScraper class, providing the CSV file name as an argument
scraper = ArticleScraper(csv_file)

# Call the scrape_articles() method of the ArticleScraper instance to initiate the scraping process
scraper.scrape_articles()


**Daily Times News**

In [ ]:
# https://dailytimes.com.pk/opeds/
# https://dailytimes.com.pk/opeds/page/2160/

# https://dailytimes.com.pk/editorial/
# https://dailytimes.com.pk/editorial/page/1023/

# https://dailytimes.com.pk/perspectives/
# https://dailytimes.com.pk/perspectives/page/357/

# https://dailytimes.com.pk/letters/
# https://dailytimes.com.pk/letters/page/1346/

# https://dailytimes.com.pk/blog/pakistan-blog/
# https://dailytimes.com.pk/blog/pakistan-blog/page/259/

# https://dailytimes.com.pk/blog/world-blog/
# https://dailytimes.com.pk/blog/world-blog/page/44/


In [36]:
class DawnNews:
    def __init__(self, url, headers):
        self.url = url  # Assigning the URL of the DawnNews website to the object's url attribute
        self.headers = headers  # Assigning the headers to be used for making HTTP requests to the object's headers attribute
        # self.driver = webdriver.Chrome()  # Initializing a Chrome webdriver instance
        
    def parse_articles(self):
        try:
            response = requests.get(self.url, headers=self.headers)  # Sending an HTTP GET request to the provided URL with the specified headers
            # print(response.content)
            response.raise_for_status()  # Checking for any HTTP request errors
            soup = BeautifulSoup(response.text, 'lxml')  # Creating a BeautifulSoup object to parse the HTML content of the response
            articles = (soup.find('ul', {'class': 'list-group'}))  # Finding all articles on the page with the specified class
            all_li = articles.find_all('li') # Finding all articles
            # print(all_li) # Printing)
            editorials_list = []
            
            for article in all_li:  # Iterating over each article found
                universityLink = article.find('div', {'class': 'col-md-12'}).find('a')['href']  # Extracting the link of the columnist
                universityTitle = article.find('div', {'class': 'col-md-12'}).find('a').text  # Extracting the title of the article
                universityCity = article.find('i', {'class': 'fa fa-map-marker'}).nextSibling.strip()  # Extracting the link of the article
                
                # Printing out the extracted information for each article
                print(universityTitle, universityCity, universityLink)
                
                editorials_list.append({
                    'universityTitle': universityTitle,
                    # 'universityLink': universityLink,
                    'universityCity': universityCity,
                })
                
                
            # Convert the list of dictionaries to a DataFrame
            df = pd.DataFrame(editorials_list)
            
            # Save DataFrame to CSV file
            df.to_csv('Ireland_Universities.csv', index=False)
                
        
        except requests.exceptions.RequestException as e:  # Catching any HTTP request exceptions
            print(f'Error: {e}')  # Printing out the error message if an exception occurs
        

In [75]:
DawnObject = DawnNews(  # Creating an instance of the DawnNews class with the specified parameters
                        'http://study.eu/search?countries=ie&page=20&sort=recommended',  # Providing the URL of the DawnNews website to scrape
                        headers={  # Providing the headers to be used for making HTTP requests
                              'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',  # User-Agent header to simulate a web browser
                              'Accept-Language': 'en-US,en;q=0.9',  # Accept-Language header to specify preferred languages for content
                              'Accept-Encoding': 'gzip, deflate, br',  # Accept-Encoding header for specifying acceptable content encoding
                              'Connection': 'keep-alive',  # Connection header to keep the connection open
                        }
                  )


In [76]:
DawnObject.parse_articles()  # Calling the parse_articles method of the DawnObject instance
# DawnObject.scrap_columns_section_list()  # This line is commented out and not executed because the method is not implemented yet

University College Cork (UCC) Cork, Ireland /university/university-college-cork-ucc
University of Galway Galway, Ireland /university/university-of-galway
University College Cork (UCC) Cork, Ireland /university/university-college-cork-ucc
Dublin City University (DCU) Dublin, Ireland /university/dublin-city-university-dcu
TU Shannon Athlone, Ireland /university/tu-shannon
Athlone Institute of Technology (AIT) Athlone, Ireland /university/athlone-institute-of-technology-ait
Dublin City University (DCU) Dublin, Ireland /university/dublin-city-university-dcu
University College Cork (UCC) Cork, Ireland /university/university-college-cork-ucc
TU Shannon Athlone, Ireland /university/tu-shannon
Athlone Institute of Technology (AIT) Athlone, Ireland /university/athlone-institute-of-technology-ait


In [ ]:
# Navigate to the website
# driver.get('https://www.dawn.com/newspaper/column')

# # Scroll and extract data
# scroll_pause_time = 2  # Adjust as needed
# last_height = driver.execute_script("return document.body.scrollHeight")

# while True:
#     # Scroll down to bottom
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     # Wait to load page
#     time.sleep(scroll_pause_time)

#     # Calculate new scroll height and compare with last scroll height
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:
#         break
#     last_height = new_height

# # Extract data from the page
# columns = driver.find_elements_by_xpath('//h2[@class="story__title"]/a')

# # Print titles of columns
# for column in columns:
#     print(column.text)

# # Close the WebDriver
# driver.quit()


In [1]:
# git rebase --abort   --continue --skip
# git rebase -i HEAD~10
# esc :wq